In [1]:
import pandas as pd

Конфигурация склада

In [2]:
df = pd.read_json('files/terminal.json')

In [3]:
df =  df[df['IsDeleted'] == False]

In [4]:
df = df.drop(['Id', 'ZoneStateId', 'Is20', 'Is40', 
              'IsBuffer', 'IsFixed', 'IsPhysSection', 
              'IsFasteningNeeded', 'StormWarning', 
              '$_insert_at', '$_source_ts_ns', 
              'AutoLoaderSettingOrder', 'GantryLoaderSettingOrder', 
              'IsForDangerInspection', 'GeneralCargoStorageAreaId', 
              'AccessFromEnd', 'AccessFromStart', 'TerminalId', 
              'IsRowsInverted', 'IsSectionsInverted', 'Rotation',
            'IsDeleted', 'RfDetailsId', 'FastenedRows', 'KeepFree'], axis=1 )

In [5]:
df_terminal_config = df[['Number', 'RowMinNumber', 'RowMaxNumber', 'LayerMaxNumber', 'zone_name']]

In [6]:
# Пример создания конфигурации склада
def create_warehouse_configuration(df_terminal_config):
    warehouse = {}
    for _, row in df_terminal_config.iterrows():
        bay = row['Number']
        row_min = row['RowMinNumber']
        row_max = row['RowMaxNumber']
        tier_max = row['LayerMaxNumber']
        block = row['zone_name']
        
        for r in range(row_min, row_max + 1):
            for t in range(1, tier_max + 1):
                slot_name = f"{block} {bay} {r} {t}"
                warehouse[slot_name] = "empty"
    return warehouse

warehouse = create_warehouse_configuration(df_terminal_config)


ФИЧИ

In [7]:
category_columns = ['cnt_iso', 'is_laden', 'terminal_of_arrival', 'move_from', 'move_to',
 'move_to_direction', 'dangerous_class', 'month receive_date',
 'year receive_date', 'shipcall_receive_vessel']

Текущие ктк

In [8]:
terminal_cnt = pd.read_csv('files/screenTerminal.csv')

/var/folders/1r/cqqk6zj11qv5nqfgj28d6pzc0000gn/T/ipykernel_14041/281201257.py:1: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  terminal_cnt = pd.read_csv('files/screenTerminal.csv')


In [9]:
def update_warehouse_with_terminal(warehouse, terminal_cnt):
    for _, row in terminal_cnt.iterrows():
        location = row['working_area_location']
        container_id = row['cnt_no']
        if location in warehouse:
            warehouse[location] = container_id
    return warehouse

warehouse = update_warehouse_with_terminal(warehouse, terminal_cnt)


In [10]:
terminal_cnt['month receive_date'] = 6
terminal_cnt['year receive_date'] = 2024
terminal_cnt['terminal_of_arrival'] = 0
terminal_cnt['move_to_direction'] = 0


In [11]:
# date_columns = ['receive_date']
# for i in date_columns:
#     terminal_cnt[i] = pd.to_datetime(terminal_cnt[i], errors='coerce')
#     terminal_cnt[f'month {i}'] = terminal_cnt[i].dt.month.astype('object')
#     terminal_cnt[f'year {i}'] = terminal_cnt[i].dt.year.astype('object')
#     terminal_cnt.drop(i, axis=1, inplace=True)

In [12]:
terminal_cnt.fillna('unknown2', inplace=True)

/var/folders/1r/cqqk6zj11qv5nqfgj28d6pzc0000gn/T/ipykernel_14041/2482766985.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  terminal_cnt.fillna('unknown2', inplace=True)


In [13]:
def split_shipcall(shipcall):
    if shipcall == 'нет данных':
        return pd.Series([None, None, None])
    else:
        parts = shipcall.rsplit(' ', 1)
        vessel = parts[0]
        voyages = parts[1].split('/')
        return pd.Series([vessel, voyages[0], voyages[1]])
terminal_cnt[['shipcall_receive_vessel', 'shipcall_receive_voyage1', 'shipcall_receive_voyage2']] = terminal_cnt['shipcall_receive'].apply(split_shipcall)
terminal_cnt.drop(columns=['shipcall_receive', 'shipcall_receive_voyage1', 'shipcall_receive_voyage2'], inplace=True)

In [14]:
terminal_cnt[category_columns] = terminal_cnt[category_columns].astype('category')
terminal_cnt[category_columns] = terminal_cnt[category_columns].apply(lambda x: x.cat.codes)

Новые ктк

In [15]:
new_cnt = pd.read_json('files/newCnt.json')

In [16]:
date_columns = ['receive_date']
for i in date_columns:
    new_cnt[i] = pd.to_datetime(new_cnt[i], errors='coerce')
    new_cnt[f'month {i}'] = new_cnt[i].dt.month.astype('object')
    new_cnt[f'year {i}'] = new_cnt[i].dt.year.astype('object')
    new_cnt.drop(i, axis=1, inplace=True)

In [17]:
new_cnt.fillna('unknown2', inplace=True)

/var/folders/1r/cqqk6zj11qv5nqfgj28d6pzc0000gn/T/ipykernel_14041/401338038.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_cnt.fillna('unknown2', inplace=True)


In [18]:
def split_shipcall(shipcall):
    if shipcall == 'нет данных':
        return pd.Series([None, None, None])
    else:
        parts = shipcall.rsplit(' ', 1)
        vessel = parts[0]
        voyages = parts[1].split('/')
        return pd.Series([vessel, voyages[0], voyages[1]])
new_cnt[['shipcall_receive_vessel', 'shipcall_receive_voyage1', 'shipcall_receive_voyage2']] = new_cnt['shipcall_receive'].apply(split_shipcall)
new_cnt.drop(columns=['shipcall_receive', 'shipcall_receive_voyage1', 'shipcall_receive_voyage2'], inplace=True)

In [19]:
new_cnt['move_to'] = 0
new_cnt['move_to_direction'] = 0

In [20]:
new_cnt[category_columns] = new_cnt[category_columns].astype('category')
new_cnt[category_columns] = new_cnt[category_columns].apply(lambda x: x.cat.codes)

мапинг настроить

Прогноз

In [21]:
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score, davies_bouldin_score
import joblib
import numpy as np

In [22]:
# Функция для модели прогнозирования времени хранения
def evaluate_model(model, X_test):
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    return y_pred
# Загрузка модели и категорий
model = joblib.load('pkl/best_rf_model.pkl')
category_mappings = joblib.load('pkl/category_mappings.pkl')


In [23]:
# Проверка атрибутов модели
if hasattr(model, 'feature_names_in_'):
    feature_names = model.feature_names_in_
    print("Модель ожидает следующие атрибуты на вход:")
    print(feature_names)
else:
    print("Модель не содержит информации о именах атрибутов.")

Модель ожидает следующие атрибуты на вход:
['cnt_iso' 'is_laden' 'terminal_of_arrival' 'move_from' 'move_to'
 'move_to_direction' 'dangerous_class' 'month receive_date'
 'year receive_date' 'shipcall_receive_vessel']


category_columns = ['cnt_iso' 'is_laden' 'terminal_of_arrival' 'move_from' 'move_to'
 'move_to_direction' 'dangerous_class' 'month receive_date'
 'year receive_date' 'shipcall_receive_vessel']

Прогнозирование для новый ктк

In [24]:
X_new_cnt = new_cnt[category_columns]
new_cnt['storage_days'] = evaluate_model(model, X_new_cnt)

In [25]:
new_cnt['location'] = None

Прогнозирование для текущих ктк 

In [26]:
X_old_cnt = terminal_cnt[category_columns]
terminal_cnt['storage_days'] = evaluate_model(model, X_old_cnt) - terminal_cnt['storage_days']

In [27]:
terminal_cnt

,rep_relevance,cnt_no,cnt_iso,cnt_size,cnt_type,is_laden,terminal,working_area_location,section_liter,section_no,...,is_dangerous,is_oversized,is_reefer,is_overstayed,is_categorized,month receive_date,year receive_date,terminal_of_arrival,move_to_direction,shipcall_receive_vessel
0,2024-06-23 17:00:00.000,HJCU2184928,4,20,DRY,1,КТ,S 53 4 1,S,53.0,...,False,False,False,False,False,0,0,0,0,-1
1,2024-06-23 17:00:00.000,HJCU4209970,26,40,DRY,1,КТ,S 53 5 1,S,53.0,...,False,False,False,False,False,0,0,0,0,-1
2,2024-06-23 17:00:00.000,HJCU4930775,26,40,DRY,1,КТ,T 1 1 1,T,1.0,...,False,False,False,False,False,0,0,0,0,-1
3,2024-06-23 17:00:00.000,GLDU7135694,41,40,DRY,1,КТ,S 53 11 1,S,53.0,...,False,False,False,False,False,0,0,0,0,-1
4,2024-06-23 17:00:00.000,BSIU2239645,4,20,DRY,1,КТ,S 53 10 1,S,53.0,...,False,False,False,False,False,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17284,2024-06-23 17:00:00.000,CSKU2714717,4,20,DRY,0,УТ,8C 33 2 1,8C,33.0,...,False,False,False,False,False,0,0,0,0,6
17285,2024-06-23 17:00:00.000,DLTU9026670,45,40,REEFER,1,Буфер,нет данных,нет данных,unknown2,...,False,False,True,False,True,0,0,0,0,-1
17286,2024-06-23 17:00:00.000,GLDU5133619,4,20,DRY,0,УТ,8C 33 2 2,8C,33.0,...,False,False,False,False,False,0,0,0,0,6
17287,2024-06-23 17:00:00.000,BMOU9721294,45,40,REEFER,1,УТ,9Y 11 2 2,9Y,11.0,...,False,False,True,False,True,0,0,0,0,53


Кластеризация

ФИЧИ

In [28]:
cluster_columns = ['cnt_iso', 'is_laden', 'terminal_of_arrival', 'move_from', 'move_to',
 'move_to_direction', 'dangerous_class', 'month receive_date',
 'year receive_date', 'shipcall_receive_vessel', 'storage_days']

In [29]:
# Объединение текущих и новоприбывших контейнеров для кластеризации
combined_containers = pd.concat([new_cnt[cluster_columns], terminal_cnt[cluster_columns]], ignore_index=True)

In [30]:
# Преобразование категориальных данных в числовые значения
for col in cluster_columns[:-1]:  # Пропуская 'storage_days'
    combined_containers[col] = combined_containers[col].astype('category').cat.codes

# Нормализация атрибута 'storage_days'
scaler = StandardScaler()
combined_containers['storage_days'] = scaler.fit_transform(combined_containers[['storage_days']])


In [31]:
combined_containers

,cnt_iso,is_laden,terminal_of_arrival,move_from,move_to,move_to_direction,dangerous_class,month receive_date,year receive_date,shipcall_receive_vessel,storage_days
0,0,0,2,2,0,0,1,0,0,6,0.332693
1,1,1,2,2,0,0,1,0,0,8,0.265755
2,0,0,2,2,0,0,1,0,0,6,0.332693
3,0,0,2,2,0,0,1,0,0,6,0.332693
4,0,0,2,2,0,0,1,0,0,6,0.332693
...,...,...,...,...,...,...,...,...,...,...,...
17320,4,0,0,4,2,0,26,0,0,7,0.232286
17321,45,1,0,0,5,0,26,0,0,0,0.299224
17322,4,0,0,4,2,0,26,0,0,7,0.232286
17323,45,1,0,4,1,0,26,0,0,54,0.466569


In [32]:
warehouse

{'0B 36 1 1': 'SMGU2002353',
 '0B 36 1 2': 'SMGU2003690',
 '0B 36 2 1': 'empty',
 '0B 36 2 2': 'empty',
 '0B 34 1 1': 'empty',
 '0B 34 1 2': 'empty',
 '0B 34 2 1': 'empty',
 '0B 34 2 2': 'empty',
 '0B 33 1 1': 'FITU5622608',
 '0B 33 1 2': 'TCNU9580585',
 '0B 33 2 1': 'empty',
 '0B 33 2 2': 'empty',
 '0B 39 1 1': 'SKHU9305025',
 '0B 39 1 2': 'SKHU9314644',
 '0B 39 2 1': 'empty',
 '0B 39 2 2': 'empty',
 '0B 22 1 1': 'GLDU2299057',
 '0B 22 1 2': 'TEMU0588287',
 '0B 22 2 1': 'empty',
 '0B 22 2 2': 'empty',
 '0B 15 1 1': 'VKMU0008464',
 '0B 15 1 2': 'FESU5290040',
 '0B 15 2 1': 'empty',
 '0B 15 2 2': 'empty',
 '0B 11 1 1': 'FESU5423101',
 '0B 11 1 2': 'empty',
 '0B 11 2 1': 'empty',
 '0B 11 2 2': 'empty',
 '0B 48 1 1': 'empty',
 '0B 48 1 2': 'empty',
 '0B 48 2 1': 'empty',
 '0B 48 2 2': 'empty',
 '0B 46 1 1': 'empty',
 '0B 46 1 2': 'empty',
 '0B 46 2 1': 'empty',
 '0B 46 2 2': 'empty',
 '0B 44 1 1': 'empty',
 '0B 44 1 2': 'empty',
 '0B 44 2 1': 'empty',
 '0B 44 2 2': 'empty',
 '0B 14 1 1': 

In [33]:
from sklearn.semi_supervised import LabelPropagation

In [34]:
# Нормализация данных
scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_containers[category_columns])

In [ ]:
# Построение графа для кластеризации
G = nx.Graph()
for i in range(len(scaled_features)):
    G.add_node(i)

similarity_matrix = cosine_similarity(scaled_features)
threshold = 0.8  # Порог для создания ребер
for i in range(len(similarity_matrix)):
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix[i][j] > threshold:
            G.add_edge(i, j, weight=similarity_matrix[i][j])

In [ ]:
# Функция для подсчета числа кластеров на каждой итерации
def label_propagation_iterations(graph, max_iter=100):
    num_clusters = []
    all_communities = []
    for i in range(max_iter):
        communities = list(nx.algorithms.community.label_propagation_communities(graph))
        all_communities.append(communities)
        num_clusters.append(len(communities))
        if i < max_iter - 1:
            nx.set_node_attributes(graph, {n: com for com, community in enumerate(communities) for n in community}, 'label')
    return num_clusters, all_communities

# Применение алгоритма Label Propagation для кластеризации и получение числа кластеров по итерациям
num_clusters, all_communities = label_propagation_iterations(G, max_iter=100)

# Выбор последней итерации кластеризации
final_communities = all_communities[-1]

In [ ]:
# Присвоение кластеров контейнерам
cluster_labels = np.zeros(len(combined_containers))
for cluster_id, community in enumerate(final_communities):
    for node in community:
        cluster_labels[node] = cluster_id

combined_containers['cluster'] = cluster_labels

In [ ]:
# Определение секций для новых контейнеров на основе кластеров
new_container_sections = []
new_container_coordinates = []
for idx, row in new_cnt.iterrows():
    cluster_id = combined_containers.loc[len(terminal_cnt) + idx, 'cluster']
    possible_sections = terminal_cnt[terminal_cnt['cluster'] == cluster_id]
    
    if not possible_sections.empty:
        # Найти контейнер с большим временем хранения и минимальной разницей
        best_container = possible_sections[possible_sections['predicted_departure_days'] >= row['predicted_departure_days']].sort_values(by='predicted_departure_days', ascending=True).iloc[0]
        new_container_sections.append(best_container['location'])
        
        # Найти свободное место сверху
        current_z = terminal_cnt[(terminal_cnt['X'] == best_container['X']) & (terminal_cnt['Y'] == best_container['Y'])]['Z'].max()
        new_container_coordinates.append((best_container['X'], best_container['Y'], current_z + 1))
    else:
        new_container_sections.append('No section found')
        new_container_coordinates.append((None, None, None))

In [ ]:
new_cnt['proposed_section'] = new_container_sections
new_cnt['proposed_coordinates'] = new_container_coordinates